In [1]:
import os
from os import listdir
from os.path import isfile, join
from os import walk
from bs4 import BeautifulSoup
from precessedData import *
from time import time



In [2]:
#得到所有路径
mypath = ['1','2','3']
fs = []
for i in range(3):
    for (dirpath, dirnames, filenames) in walk(mypath[i]):
    #     print(dirpath)
    #     print(dirnames)
        for file in filenames:
            if file == 'export.css':
                continue
            fs.append(dirpath + "/" + file)
    #     break

#将文件目录写入
with open('filepath.txt', 'w') as f:
    for i in range(1,len(fs)):
        f.write(fs[i])

    f.close()
        

In [3]:
index = [] #纪录所有fields.html的下标，按文件夹计算得分
for i in range(len(fs)):
    if fs[i].find('fields.html') >= 0:
        index.append(i)

In [4]:
len(index)

3896

In [7]:
begin = time()
user_source_dict = {} # user_id : [总分，每一题的得分]
# 得到所有文件的评分
k = 0
# 得到部分文件进行测试，这里可以进行测试，但是我们的结果文件是全部的
# 测试的时候，可是设置n的值,小于长度即可

# n = len(index)
n = 5
for i in range(0,n-1):
    if i% 50 == 0: print(i) #每50个，打印计算的进度
    user_id,source = uniteAll(fs[index[i]:index[i+1]])
    
    # tfidf得到关键词
    keyWords = getQuestonKeys(fs[index[i]])
    
    #得到用户答案的LDA矩阵
    QuestonLDAMat = getQuestonLDAMat(fs[index[i]])
    
    if user_id != None:
        #这里可以设置注释，得到需要的信息
#         user_source_dict[user_id] = [sum(source),source,keyWords]
        user_source_dict[user_id] = [sum(source),source,QuestonLDAMat]
        
    k = i
    
end = time()

print("Total time(minutes):", (end - begin)/60)

0


/home/xiaoran/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/xiaoran/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))
/home/xiaoran/anaconda3/lib/python3.5/site-packages/gensim/models/ldamodel.py:525: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


Total time(minutes): 0.5015506148338318


In [8]:
# 写入文件分数和LDA矩阵
import csv
headers = ['user_id','总分','Question1分数','Question2分数','Question3分数','Question4分数','Question1 LDAMat','Question2 LDAMat','Question3 LDAMat','Question4 LDAMat']
with open('source_LDAMat.csv','w',newline="") as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for key in user_source_dict.keys():
        tmp = []
        tmp.append(key)
        tmp.append(user_source_dict[key][0])
        tmp.extend(user_source_dict[key][1])
        for d in user_source_dict[key][2]:
            tmp.append(d)
        
        f_csv.writerows([tmp])



In [7]:
# 写入文件分数和关键词
import csv
headers = ['user_id','总分','Question1分数','Question2分数','Question3分数','Question4分数','Question1KeyWords','Question2KeyWords','Question3KeyWords','Question4KeyWords']
with open('source_keywords.csv','w',newline="") as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for key in user_source_dict.keys():
        tmp = []
        tmp.append(key)
        tmp.append(user_source_dict[key][0])
        tmp.extend(user_source_dict[key][1])
        for d in user_source_dict[key][2]:
            tmp.append(d)
        
        f_csv.writerows([tmp])



In [7]:
# 写入文件,只有分数
import csv
headers = ['user_id','总分','Question1分数','Question2分数','Question3分数','Question4分数']
with open('user_source_dict.csv','w',newline="") as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    for key in user_source_dict.keys():
        tmp = []
        tmp.append(key)
        tmp.append(user_source_dict[key][0])
        tmp.extend(user_source_dict[key][1])
        f_csv.writerows([tmp])



处理结果文件，剔除含有空值的行，还有空值的原因是：有些数据不完整，例如没有评价文件，或者只有一个评分文件等
对于自动评分系统，Nan的评分，就相当与没有这个user的评分，可以直接删除，

这里几个结果文件，是分批次运行处理的，因为全部运算太耗时，大概需要4个小时左右。
我们进行的整合，并给出的测试的代码
建议选择部分文件进行测试，

In [130]:
# 处理结果文件，剔除含有空值的行，还有空值的原因是：有些数据不完整，例如没有评价文件，或者只有一个评分文件等
# 对于自动评分系统，Nan的评分，就相当与没有这个user的评分，可以直接删除，

import pandas as pd

source = pd.read_csv('user_source_dict.csv',error_bad_lines=False)
source1 = pd.read_csv('user_source_dict1.csv',error_bad_lines=False)
source2 = pd.read_csv('user_source_dict2.csv',error_bad_lines=False)
source3 = pd.read_csv('user_source_dict3.csv',error_bad_lines=False)

b'Skipping line 19: expected 6 fields, saw 7\n'


In [125]:
source.info()
source1.info()
source2.info()
source3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167 entries, 0 to 1166
Data columns (total 6 columns):
user_id        1167 non-null object
总分             1167 non-null float64
Question1分数    1167 non-null float64
Question2分数    1167 non-null float64
Question3分数    1167 non-null float64
Question4分数    1167 non-null float64
dtypes: float64(5), object(1)
memory usage: 54.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 446 entries, 0 to 445
Data columns (total 6 columns):
user_id        446 non-null object
总分             428 non-null float64
Question1分数    441 non-null float64
Question2分数    432 non-null float64
Question3分数    439 non-null float64
Question4分数    433 non-null float64
dtypes: float64(5), object(1)
memory usage: 21.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 6 columns):
user_id        116 non-null object
总分             114 non-null float64
Question1分数    115 non-null float64
Question2分数    114 non-null float64
Q

In [131]:
source = source.append(source1)
source = source.append(source2)
source = source.append(source3)
# source.to_csv('user_source_dict.csv')
#删除空值，写入最后的文件
source = source.dropna()
source.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3631 entries, 0 to 2031
Data columns (total 6 columns):
user_id        3631 non-null object
总分             3631 non-null float64
Question1分数    3631 non-null float64
Question2分数    3631 non-null float64
Question3分数    3631 non-null float64
Question4分数    3631 non-null float64
dtypes: float64(5), object(1)
memory usage: 198.6+ KB


In [132]:
source.to_csv('FinallySource.csv')

,Question1分数,Question2分数,Question3分数,Question4分数,Unnamed: 0,Unnamed: 0.1,user_id,总分
0,0.331941,3.570770,3.228145,0.392510,0.0,2.0,567cdbfc3a9a40e9a4d52c0d2213262b13798063,7.523367
1,0.818534,0.000000,0.955046,0.442489,1.0,3.0,c5f457c5e8359c8bb98c636ce659b5e608177dab,2.216069
2,0.862994,1.368206,1.490147,0.441602,2.0,4.0,2a4428c22bc0baad6d2e0ef96ff7dced51034761,4.162948
3,1.778944,0.824958,0.914853,0.790504,3.0,5.0,fed728ca81f70c75ff59a3d2ad30d68db2be4a5b,4.309260
4,0.742453,0.000000,2.930765,1.782663,4.0,6.0,b3b6cc01baf323378b437ec1818f747ceefec0a9,5.455881
5,0.333333,2.005181,1.968865,1.671100,5.0,7.0,62d1a78e94b3b21b55a8da0e23149b6b737907f5,5.978480
6,0.857856,0.968234,3.476990,2.154324,6.0,8.0,0f1f34007dcaace17314c5f1943ffa3cd9e6f2c1,7.457404
7,0.523087,0.000000,4.275843,0.965926,7.0,9.0,04675c2640658f3ca2667b14181a11209dfd8ac3,5.764855
8,0.342997,2.490148,4.176500,2.302299,8.0,10.0,9ed78e87aafcbe898c06d2ed13dfd36379320de6,9.311944
9,1.109777,2.305252,1.546390,1.989600,9.0,11.0,32fadb36686ca4a331a4d98b09ba5781f5291a56,6.951020
